## BASELINE CNN
- 思路：先做Jupyterbooks，有了运行结果后放进class跑。11号今天写CNN并跑出基本结果。
- train (5509260, 14) , validation (1836420, 14) , test (1836420, 14)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf

%matplotlib inline
plt.style.use('ggplot')


In [2]:
train_path = '../src/train.csv'
valid_path = '../src/validation.csv'
test_path = '../src/test.csv'
combined_path = '../src/combined_file.csv'

In [3]:
def read_data(file_path):
    # column_names = ['subject','activity','time_s', 'lw_x','lw_y','lw_z','lh_x','lh_y','lh_z','la_x','la_y','la_z','ra_x','ra_y','ra_z']
    data = pd.read_csv(file_path,header = 0)
    return data
# def feature_normalize(dataset):
#     mu = np.mean(dataset,axis = 0)
#     sigma = np.std(dataset,axis = 0)
#     return (dataset - mu)/sigma

def plot_axis(ax, x, y, title):
    ax.plot(x, y)
    ax.set_title(title)
    ax.xaxis.set_visible(False)
    ax.set_ylim([min(y) - np.std(y), max(y) + np.std(y)])
    ax.set_xlim([min(x), max(x)])
    ax.grid(True)
    
def plot_activity(activity,data):
    fig, (ax0, ax1, ax2, ax3) = plt.subplots(nrows = 4, figsize = (15, 10), sharex = True)
    plot_axis(ax0, data['time_s'], data['magnitude_lw'], 'magnitude_lw')
    plot_axis(ax1, data['time_s'], data['magnitude_lh'], 'magnitude_lh')
    plot_axis(ax2, data['time_s'], data['magnitude_la'], 'magnitude_la')
    plot_axis(ax3, data['time_s'], data['magnitude_ra'], 'magnitude_ra')
    plt.subplots_adjust(hspace=0.2)
    fig.suptitle(activity)
    plt.subplots_adjust(top=0.90)
    plt.show()


In [4]:
train_df = read_data(train_path)


In [5]:
# for activity in np.unique(train_df["activity"]):
#     subset = train_df[train_df["activity"] == activity][:180]
#     # plot_activity(activity,subset)

In [6]:
magnitude_columns = ['magnitude_lw', 'magnitude_lh', 'magnitude_la', 'magnitude_ra']
activity_column = 'activity'

# Get the magnitude data
magnitude_data = train_df[magnitude_columns].values

# Get the activity data
activity_data = train_df[activity_column].values

# Concatenate magnitude and activity data along the channel axis
input_data = np.concatenate([magnitude_data, activity_data[:, np.newaxis]], axis=-1)

num_channels = magnitude_data[0].shape[-1]
print("Number of input channels:", num_channels)


print('magnitude_data.shape,',magnitude_data.shape)
# Determine the shape of the magnitude data
H, W = magnitude_data.shape


N = 64
# # Reshape input data to the expected shape (batch_size, height, width, channels)
# input_data = input_data.reshape(N, H, W, -1)

# # Convert input data to a TensorFlow tensor
input_tensor = tf.constant(input_data, dtype=tf.float32)


Number of input channels: 4
magnitude_data.shape, (11018034, 4)


In [7]:
import tensorflow as tf

class CNNModel(tf.Module):
    def __init__(self,channels,height,width,num_classes):
        self.conv1_weights = tf.Variable(tf.random.normal([3, 3, channels, 32]))
        self.conv1_biases = tf.Variable(tf.zeros([32]))
        self.conv2_weights = tf.Variable(tf.random.normal([3, 3, 32, 64]))
        self.conv2_biases = tf.Variable(tf.zeros([64]))
        self.fc1_weights = tf.Variable(tf.random.normal([height // 4 * width // 4 * 64, 64]))
        self.fc1_biases = tf.Variable(tf.zeros([64]))
        self.fc2_weights = tf.Variable(tf.random.normal([64, num_classes]))
        self.fc2_biases = tf.Variable(tf.zeros([num_classes]))
        self.height = height
        self.width = width


    def __call__(self, inputs):
        x = tf.nn.conv2d(inputs, self.conv1_weights, strides=[1, 1, 1, 1], padding='SAME')
        x = tf.nn.relu(tf.nn.bias_add(x, self.conv1_biases))
        x = tf.nn.max_pool2d(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

        x = tf.nn.conv2d(x, self.conv2_weights, strides=[1, 1, 1, 1], padding='SAME')
        x = tf.nn.relu(tf.nn.bias_add(x, self.conv2_biases))
        x = tf.nn.max_pool2d(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

        x = tf.reshape(x, [-1, self.height // 4 * self.width // 4 * 64])
        x = tf.nn.relu(tf.matmul(x, self.fc1_weights) + self.fc1_biases)

        output = tf.matmul(x, self.fc2_weights) + self.fc2_biases
        return output





In [8]:
# Instantiate the model
model = CNNModel(4,H,W,6)
# # Pass the input tensor through the model
output = model(input_tensor)

: 

: 